In [1]:
"""
AI Fashion Model Visualization Tool – MVP reference implementation
Author : You
Date   : 21-Oct-2025
Hardware: NVIDIA GPU ≥8 GB VRAM (RTX 3060 / T4)
Runtime : ±5 s / image @ 512×512
"""

# ------------------ 1. One-shot install ------------------
# !pip install -q diffusers==0.30.0 transformers accelerate xformers opencv-python controlnet_aux safetensors torch --upgrade

# ------------------ 2. Imports ------------------
import torch, cv2, numpy as np, os, gradio as gr
from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, StableDiffusionControlNetInpaintPipeline
from controlnet_aux import OpenposeDetector
from PIL import Image
import warnings, gc, time
warnings.filterwarnings("ignore")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE  = torch.float16 if DEVICE=="cuda" else torch.float32

# ------------------ 3. Download weights (cached) ------------------
base_id   = "runwayml/stable-diffusion-inpainting"   # public SD 1.5 inpainting
ctrl_id   = "lllyasviel/control_v11p_sd15_openpose"
lora_path = "https://huggingface.co/your-org/fashion-lora/resolve/main/fashion-lora.safetensors"  # <-- replace with your LoRA

# ------------------ 4. Load models once ------------------
print("Loading models …")
controlnet = ControlNetModel.from_pretrained(ctrl_id, torch_dtype=DTYPE).to(DEVICE)
pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
        base_id, controlnet=controlnet, torch_dtype=DTYPE, safety_checker=None)
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()          # low VRAM friendly
if DEVICE=="cuda": pipe.enable_attention_slicing(1)

# Optional – load LoRA ( CivitAI or your own )
# pipe.load_lora_weights(lora_path, adapter_name="fashion")
# pipe.set_adapters(["fashion"], adapter_weights=[1.0])

openpose = OpenposeDetector.from_pretrained("lllyasviel/Annotators")

# ------------------ 5. Core try-on logic ------------------
def auto_mask(garment_flat:Image)->Image:
    """Very crude garment mask via HSV saturation – replace with U²-Net later."""
    im = np.array(garment_flat)
    hsv = cv2.cvtColor(im, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, (0,50,50), (180,255,255))
    mask = cv2.GaussianBlur(mask, (11,11), 0)
    return Image.fromarray(mask)

def try_on(garment_path:str, person_path:str, guidance=7.5, steps=25, seed=42):
    start = time.time()
    garment = Image.open(garment_path).convert("RGB").resize((512,512))
    person  = Image.open(person_path).convert("RGB").resize((512,512))
    mask    = auto_mask(garment).resize((512,512))

    pose_img = openpose(person).resize((512,512))

    generator = torch.Generator(DEVICE).manual_seed(seed)
    result = pipe(
        prompt="fashion model wearing the garment, high quality, detailed fabric",
        negative_prompt="blurry, lowres, distorted face, extra limbs",
        image=person,
        mask_image=mask,
        control_image=pose_img,
        num_inference_steps=steps,
        guidance_scale=guidance,
        generator=generator,
        width=512,
        height=512
    ).images[0]

    print(f"DONE in {time.time()-start:.1f}s")
    return result

# ------------------ 6. Gradio UI (optional) ------------------
demo = gr.Interface(
    fn=try_on,
    inputs=[gr.Image(type="filepath", label="Flat-lay garment"),
            gr.Image(type="filepath", label="Customer photo")],
    outputs=gr.Image(type="pil", label="AI Try-On"),
    title="AI Fashion Model Visualization Tool – MVP",
    examples=[["./sample_garment.jpg", "./sample_person.jpg"]]
)

# ------------------ 7. CLI entry ------------------
if __name__ == "__main__":
    import fire
    fire.Fire(try_on)   # python ai_fashion_tryon.py garment.jpg person.jpg

ModuleNotFoundError: No module named 'controlnet_aux'